# Teague Stotlar

## Web Scraping for 2013, 2018, 2023

In [6]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import time
import random
import pandas as pd

BASE_DOMAIN = "https://iowacity.iowaassessors.com/"
search_url = "https://iowacity.iowaassessors.com/showResSaleSearch.php?"

# I am running on Chrome version 134
options = uc.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

# Scrape data for a given year
def scrape_year_data(year, driver, wait):
    print(f"\n--- Starting scrape for year {year} ---\n")
    
    # Go to search page
    driver.get(search_url)
    time.sleep(3)  
    
    # Date Range
    sale_date_from = wait.until(EC.presence_of_element_located((By.NAME, "sale_date1")))
    sale_date_from.clear()
    sale_date_from.send_keys(f"01/01/{year}") 
    
    sale_date_to = driver.find_element(By.NAME, "sale_date2")
    sale_date_to.clear()
    sale_date_to.send_keys(f"12/30/{year}") 
    
    # Make the occupancy single family / owner occupied (1)
    occupancy_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[9]/div[2]/select")
    Select(occupancy_select).select_by_value("1")
    
    # Normal values (0)
    transaction_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[8]/div[2]/select")
    Select(transaction_select).select_by_value("0")
    
    # Residential Class (2)
    class_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[23]/div[2]/select")
    Select(class_select).select_by_value("2")
    
    # Display Results Button
    display_results_button = driver.find_element(By.ID, "display_results")
    display_results_button.click()
    
    # Wait for Results to load up
    def find_results_table():
        try:
            return wait.until(EC.presence_of_element_located((By.ID, "tblSearchResults")))
        except:
            try:
                return wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//table[.//th[contains(text(), 'Sale Date')]]")
                ))
            except Exception as e:
                print(f"Results table not for year {year}.")
                return None
    
    results_table = find_results_table()
    if not results_table:
        print(f"Results table not found for year {year}.")
        return []
    
    time.sleep(5) 
    
    # Scrape Results
    all_data = []
    page_num = 1
    
    # Get the headers
    header_row = results_table.find_elements(By.XPATH, ".//th")
    header_texts = [header.text.strip() for header in header_row]
    print(f"Headers on page for year {year}:", header_texts)
    
    while True:
        print(f"Scraping page {page_num} for year {year}...")
        results_table = find_results_table()
        if not results_table:
            print(f"Results table not found on page {page_num} for year {year}. Exiting loop.")
            break
        
        rows = results_table.find_elements(By.XPATH, ".//tr")
        
        # Debugging: print first data row cell
        if page_num == 1 and len(rows) > 1:
            first_row_cells = rows[1].find_elements(By.TAG_NAME, "td")
            print(f"\nDEBUG: First data row cell values for year {year}:")
            for idx, cell in enumerate(first_row_cells):
                print(f"  Column {idx}: '{cell.text}'")
            print()
        
        # Extract data from the rows, starting from index 1 to skip header
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if cells and len(cells) >= 11:
                # Use the actual table structure based on your provided XPATHs
                record = {
                    "Parcel Number":    cells[0].text,  # First column
                    "Sale Date":        cells[1].text,  # Second column
                    "Sale Amount":      cells[2].text,  # Third column
                    "Address":          cells[4].text,  # Fifth column (skipping Photo column)
                    "Style":            cells[5].text,  # Sixth column
                    "Year Built":       cells[6].text,  # Seventh column
                    "Total SF":         cells[7].text,  # Eighth column
                    "Lot Area":         cells[8].text,  # Ninth column
                    "Appraised Value":  cells[9].text,  # Tenth column
                    "Recording":        cells[10].text  # Eleventh column
                }
                all_data.append(record)
        
        # Find Next Page link
        next_links = driver.find_elements(By.XPATH, "//a[./img[contains(normalize-space(@alt), 'Next Page')]]")
        print(f"Found {len(next_links)} Next Page link(s) on page {page_num} for year {year}.")
        
        if next_links:
            next_href = next_links[0].get_attribute("href")
            if not next_href:
                print(f"Next page link found but href is empty for year {year}. Ending pagination.")
                break
            full_next_url = urljoin(BASE_DOMAIN, next_href)
            print(f"Navigating to next page for year {year}: {full_next_url}")
            driver.get(full_next_url)
            page_num += 1
            time.sleep(random.uniform(3, 5))
        else:
            print(f"No 'Next Page' link found for year {year}. Assuming last page reached.")
            break
    
    return all_data

# Main execution
try:
    driver = uc.Chrome(version_main=134, options=options)
    wait = WebDriverWait(driver, 15)
    
    # Years to scrape
    years_to_scrape = [2023, 2018, 2013]
    
    # Column order for CSV
    columns_order = [
        "Parcel Number", "Sale Date", "Sale Amount", "Address", "Style", 
        "Year Built", "Total SF", "Lot Area", "Appraised Value", "Recording"
    ]
    
    # Scrape data each year
    for year in years_to_scrape:
        # Scrape data current year
        year_data = scrape_year_data(year, driver, wait)
        
        # Create DF with scraped data
        df = pd.DataFrame(year_data)
        
        # Make sure columns are in the right order
        df = df[columns_order]
        
        # Save to CSV
        csv_filename = f"historical_transactions_{year}.csv"
        df.to_csv(csv_filename, index=False)
        
        print(f"\nScraped {len(year_data)} records for year {year}.")
        print(f"Data saved to {csv_filename}")
        print(f"Sample data for {year}:")
        print(df.head())
        
        # Add a short delay to make sure code runs smoothly
        if year != years_to_scrape[-1]:
            time.sleep(random.uniform(5, 8))
    
    print("\nAll scraping completed successfully!") # Makes output look cleaner

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if 'driver' in locals():
        driver.quit()
    print("Driver closed.")


--- Starting scrape for year 2023 ---

Headers on page for year 2023: ['Parcel Number', 'Sale Date', 'Sale Amount', 'Photo', 'Address', 'Style', 'Year Built', 'Total SF', 'Lot Area', 'Appraised Value', 'Recording']
Scraping page 1 for year 2023...

DEBUG: First data row cell values for year 2023:
  Column 0: '1014355009'
  Column 1: '05/25/2023'
  Column 2: '$60,000'
  Column 3: ''
  Column 4: '1521 BROADWAY ST, IOWA CITY'
  Column 5: '1 Story Frame'
  Column 6: '1954'
  Column 7: '1204'
  Column 8: '6,250'
  Column 9: '$150,000'
  Column 10: '6482-625'

Found 1 Next Page link(s) on page 1 for year 2023.
Navigating to next page for year 2023: https://iowacity.iowaassessors.com/results.php?page=2&history=-2&ts=1743623335
Scraping page 2 for year 2023...
Found 1 Next Page link(s) on page 2 for year 2023.
Navigating to next page for year 2023: https://iowacity.iowaassessors.com/results.php?page=3&history=-3&ts=1743623335
Scraping page 3 for year 2023...
Found 1 Next Page link(s) on page 

## Integrating the 3 datasets

In [14]:
df_2013 = pd.read_csv('historical_transactions_2013.csv')
df_2018 = pd.read_csv('historical_transactions_2018.csv')
df_2023 = pd.read_csv('historical_transactions_2023.csv')

# Add a 'Year' column
df_2013['Source Year'] = 2013
df_2018['Source Year'] = 2018
df_2023['Source Year'] = 2023

# Combine
combined_df = pd.concat([df_2013, df_2018, df_2023], ignore_index=True)

# Save
combined_df.to_csv('iowa_city_housing_combined.csv', index=False)

5. Merge in ACS County‐Level Summaries

In [1]:
import pandas as pd

files = [
  "ACS_B19013_JohnsonCo_2013.csv",
  "ACS_B19013_JohnsonCo_2018.csv",
  "ACS_B19013_JohnsonCo_2023.csv",
]
dfs = []
for f in files:
    df = pd.read_csv(f)
    # The CSV will have columns ["Year","GEO_ID","NAME","B19013_001E","B19013_001M"]
    year = int(f.split("_")[-1].split(".")[0])
    df = df[["B19013_001E"]].rename(columns={"B19013_001E":"Median_Household_Income"})
    df["Year"] = year
    dfs.append(df[["Year","Median_Household_Income"]])
income = pd.concat(dfs, ignore_index=True)
income.to_csv("ACS_B19013_JohnsonCo.csv", index=False)

In [5]:
import pandas as pd

files = [
    "ACS_B25106_JohnsonCo_2013.csv",
    "ACS_B25106_JohnsonCo_2018.csv",
    "ACS_B25106_JohnsonCo_2023.csv",
]

all_years = []
for fn in files:
    # 1) Load, skipping the first “code” header row
    df = pd.read_csv(fn, skiprows=1)
    
    # 2) Clean up your column names (strip whitespace & any trailing colon)
    df.columns = df.columns.str.strip().str.rstrip(":")
    
    # 3) Pull the year out of the filename and store it
    year = int(fn.split("_")[-1].split(".")[0])
    df["Year"] = year
    
    # 4) Total units = the single column called exactly "Estimate!!Total"
    if "Estimate!!Total" not in df.columns:
        raise KeyError(f"Expected 'Estimate!!Total' in {fn} but got: {df.columns.tolist()}")
    df["Total_Units"] = pd.to_numeric(df["Estimate!!Total"], errors="coerce")
    
    # 5) Cost-burden units = sum of every column whose name contains "30 percent or more"
    burden_cols = [c for c in df.columns if "30 percent or more" in c]
    if not burden_cols:
        raise KeyError(f"No cost-burden columns found in {fn}")
    # ensure they're numeric before summing
    df[burden_cols] = df[burden_cols].apply(pd.to_numeric, errors="coerce")
    df["Cost_Burden_Units"] = df[burden_cols].sum(axis=1)
    
    # keep only the bits we need
    all_years.append(
        df[[
            "Geography",
            "Geographic Area Name",
            "Year",
            "Total_Units",
            "Cost_Burden_Units"
        ]]
    )

# stitch all three years together
result = pd.concat(all_years, ignore_index=True)

# (optional) compute a proportion column
result["Cost_Burden_Proportion"] = (
    result["Cost_Burden_Units"] / result["Total_Units"]
)

# save
result.to_csv("ACS_B25106_JohnsonCo.csv", index=False)
print("✔ Written ACS_B25106_JohnsonCo.csv:", result.shape)


✔ Written ACS_B25106_JohnsonCo.csv: (3, 6)


In [7]:
import pandas as pd

# Build the median‐income table
income_files = [
    "ACS_B19013_JohnsonCo_2013.csv",
    "ACS_B19013_JohnsonCo_2018.csv",
    "ACS_B19013_JohnsonCo_2023.csv",
]
medians = []

for fn in income_files:
    # skip the code row
    df = pd.read_csv(fn, skiprows=1)
    # clean up trailing colons, whitespace
    df.columns = df.columns.str.strip().str.rstrip(":")
    
    # infer year
    year = int(fn.split("_")[-1].split(".")[0])
    df["Year"] = year
    
    # find the one median‐income column
    median_cols = [c for c in df.columns if c.startswith("Estimate!!Median household income")]
    if not median_cols:
        raise KeyError(f"No median‐income column found in {fn}. Available columns:\n{df.columns.tolist()}")
    median_col = median_cols[0]
    
    # convert to numeric
    df["Median_Income"] = pd.to_numeric(df[median_col], errors="coerce")
    
    # keep only what we need
    medians.append(
        df[
            ["Geography", "Geographic Area Name", "Year", "Median_Income"]
        ]
    )

df_medians = pd.concat(medians, ignore_index=True)
df_medians.to_csv("ACS_B19013_JohnsonCo.csv", index=False)
print("✔ Written ACS_B19013_JohnsonCo.csv:", df_medians.shape)


✔ Written ACS_B19013_JohnsonCo.csv: (3, 4)


In [8]:
# Merge cost‐burden + median‐income ACS tables
df_burden = pd.read_csv("ACS_B25106_JohnsonCo.csv")
df_acs    = pd.merge(
    df_burden,
    df_medians,
    on=["Geography","Geographic Area Name","Year"],
    how="left"
)
df_acs.to_csv("ACS_JohnsonCo_combined.csv", index=False)
print("✔ Written ACS_JohnsonCo_combined.csv:", df_acs.shape)

# Merge ACS into assessor transactions
df_tx = pd.read_csv("iowa_city_housing_combined.csv").rename(columns={"Source Year":"Year"})
master = pd.merge(df_tx, df_acs, on="Year", how="left")
master.to_csv("master_housing_plus_ACS.csv", index=False)
print("Written master_housing_plus_ACS.csv:", master.shape)

✔ Written ACS_JohnsonCo_combined.csv: (3, 7)
✔ Written master_housing_plus_ACS.csv: (1785, 17)


In [9]:
import pandas as pd

master = pd.read_csv("master_housing_plus_ACS.csv")
print(master.head())
print(master.dtypes)
print(master.isna().sum())

   Parcel Number   Sale Date Sale Amount                       Address  \
0     1010180001  08/01/2013     $50,000     427 N DODGE ST, IOWA CITY   
1     1014158001  01/25/2013     $65,000          1731 E ST, IOWA CITY   
2     1014306005  02/27/2013     $83,000   1120 FRANKLIN ST, IOWA CITY   
3     1016403018  08/23/2013     $85,000    316 DOUGLASS CT, IOWA CITY   
4     1014456009  11/25/2013     $86,000  1809 DEFOREST AVE, IOWA CITY   

               Style  Year Built  Total SF Lot Area Appraised Value Recording  \
0      2 Story Frame        1880      1188    6,000        $152,790  5141-586   
1      2 Story Frame        1920       880    3,325        $199,300  5043-723   
2      1 Story Frame        1935       699    8,400        $173,330  5057-663   
3      1 Story Frame        1956       768    5,100        $150,860  5153-513   
4  Split Foyer Frame        1964       960   12,138        $245,930  5184-107   

   Year       Geography  Geographic Area Name  Total_Units  Cost_Bur